In [ ]:
# In this function the input was the load of network in pandapower form (df) and the percentage of three-phase power points, rooftop PV 
# installation, and EV. Also user must give the number of loads where can be installed in feeder (Total_loads,of_feeder). 
# Finally feeder indicate the string that is included in buses where loads can be insatlled (e.g 'feeder1').

def assign_phase_type(df, three_phase_percentage, PV_percentage, EV_percentage,Total_loads_of_feeder,feeder,values_one_phase,probabilities_one_phase,values_three_phase,probabilities_three_phase):
    
    # This take the buses where the loads can be installed. Only those which name include the feeder(input variable) string
    f_df = df[df['name'].str.contains(feeder)][['bus']].copy() 
    f_df['Phase'] = 'None' # Assign to all buses the None
    
    total_avail_loads=len(f_df) # Maximum number of load buses 
    # If the load buses is less than the feeder input load number then the loads will be equal to the load buses to avoide errors
    Total_loads_of_feeder = min(Total_loads_of_feeder,total_avail_loads) 
    # Determine the number of 3-phase loads
    num_three_phase = int(three_phase_percentage * Total_loads_of_feeder)
    num_single_phase = Total_loads_of_feeder - num_three_phase # The rest number of loads was 1-phase power points
    num_none_phase = total_avail_loads - num_single_phase - num_three_phase # The rest load buses will not have any power supply
    
    # assign loads based on number of each power point type
    assignments = ['3-phase'] * num_three_phase + ['1-phase'] * num_single_phase + ['None'] * num_none_phase
    # Shuffle the DataFrame to randomly assign the 3-phase and 1-phase loads
    np.random.shuffle(assignments)
    f_df['Type']=assignments
    
    # For single-phase loads, assign 33% to phase A, 33% to phase B, and 33.3% to phase C
    
    # First find in want indices was each power supply point
    single_phase_indices = f_df[f_df['Type'] == '1-phase'].index
    three_phase_indices = f_df[f_df['Type'] == '3-phase'].index

    num_phase_A = num_phase_B = round(num_single_phase/3)
    num_phase_C = num_single_phase - (num_phase_A + num_phase_B)  # Remaining loads go to phase C
    
    assignments = ['A'] * num_phase_A + ['B'] * num_phase_B + ['C'] * num_phase_C
    # Shuffle the assignments array to ensure random distribution
    np.random.shuffle(assignments) 
    
    # Assign the phase to single-phase loads and in three phase the string 'A-B-C'
    f_df.loc[single_phase_indices, 'Phase'] = assignments[:len(single_phase_indices)]
    f_df.loc[three_phase_indices, 'Phase'] = 'A-B-C'

    # Assign randomly which buses will have rooftop PV
    num_pvs = round(Total_loads_of_feeder * PV_percentage)
    assignments = ['Yes'] * num_pvs + ['No'] *(Total_loads_of_feeder - num_pvs)
    # Shuffle the assignments array to ensure random distribution
    np.random.shuffle(assignments) 
    f_df['PV'] = 'None' # Firsly assign in all of them the 'None'
    # Then only in the buses that are having power supply point assign yes or no if have PV.
    f_df.loc[f_df['Phase'] != 'None', 'PV'] = assignments

    # This part will be replaced with randomly select max installed power based on distribution (statistic data) 
    #Initialize all as none
    f_df['PV_Power'] = 'None'
    
    #Based on power supply point type use the corresponding distribution
    f_df.loc[(f_df['Type'] == '3-phase') & (f_df['PV'] == 'Yes'), 'PV_Power'] = np.random.choice(values_three_phase, size=len(f_df.loc[(f_df['Type'] == '3-phase') & (f_df['PV'] == 'Yes'), 'PV_Power']) ,p=probabilities_three_phase)
    f_df.loc[(f_df['Type'] == '1-phase') & (f_df['PV'] == 'Yes'), 'PV_Power'] = np.random.choice(values_one_phase, size=len(f_df.loc[(f_df['Type'] == '1-phase') & (f_df['PV'] == 'Yes'), 'PV_Power']) , p=probabilities_one_phase)
    f_df.loc[(f_df['PV'] == 'No') , 'PV_Power'] = 0
    
    #Randomly assigned EV in the valid load buses with the same logic as the PVs
    num_EV = round(EV_percentage * Total_loads_of_feeder)
    assignments = ['Yes'] * num_EV + ['No'] * (Total_loads_of_feeder-num_EV)
    np.random.shuffle(assignments) 
    f_df['EV'] = 'None'
    f_df.loc[f_df['Type'] != 'None', 'EV'] = assignments
    f_df['EV_Power'] = 'None'
    
    # If 3-Phase power point was use 11kW charger used otherwise 3.5kW
    f_df.loc[(f_df['Type'] == '3-phase') & (f_df['EV'] == 'Yes'), 'EV_Power'] = 11
    f_df.loc[(f_df['Type'] == '1-phase') & (f_df['EV'] == 'Yes'), 'EV_Power'] = 3.5
    f_df.loc[(f_df['EV'] == 'No'), 'EV_Power'] = 0

    return f_df

In [ ]:
# Type of Day currently was not used however we have to made a modification to include them

# This function assign randomly and uniformly the load profile to the loads
# feederloads was the list of loads with buses and more details which produce by assign_phase_type function and 
# load profile was a list with all avaialble 24 hours load profiles
def Basic_Load_profile(feederloads,Type_of_Day,load_profile):
    
    num_of_Loads = (feederloads['Phase']!='None').sum() # Find the number of load which is connected at 

    # Define the maximum indices
    max_i = len(load_profile)  # Maximum index for customers (0 to 119)
    max_j = len(load_profile[0])  # Maximum index for days (0 to 364)
    
    # Create a time index for 24 hours with 30-minute intervals
    time_index = pd.date_range(start='00:00', end='23:30', freq='30min')

    # Initialize a DataFrame to hold the selected loads
    load_df = pd.DataFrame(index=time_index, columns=[f'Customer_{i+1}' for i in range(num_of_Loads)])
    
    for i in range(num_of_Loads):
        # Randomly select customer index i and day index j
        random_i = np.random.randint(0, max_i)  # Random integer betweenf 0 and max_i
        random_j = np.random.randint(0, max_j)  # Random integer between 0 and max_j
        selected_load_profile = load_profile[random_i, random_j]
        load_df.loc[:, load_df.columns[i]] = selected_load_profile
    load_df.index=load_df.index.time
    return load_df

In [ ]:
# # This function finally was not used however i keep it 

# def target_load_def(month_number,pv_profile,f1,f2,f3,target):
#     pv_production=0    
#     # pv_profile=PV_profile.iloc[:,month_number]
#     for pv in f1[f1['PV_Power']!='None']['PV_Power']:
#         pv_production += pv_profile * pv
#     for pv in f2[f2['PV_Power']!='None']['PV_Power']:
#         pv_production += pv_profile * pv
#     for pv in f3[f3['PV_Power']!='None']['PV_Power']:
#         pv_production += pv_profile * pv
#     return target['P_Sum']+pv_production

In [ ]:
# The target of this function was to modify the load profiles to meet the total target load considering that there was a loss
# coefficient  
def Basic_Load_modification(F1,F2,F3,target,loss_coeff):
    Initial_load = (F1.sum(axis=1) + F2.sum(axis=1) + F3.sum(axis=1))
    correction_factor = (1-loss_coeff)*target/Initial_load
    
    F1=F1.mul(correction_factor, axis=0)
    F2=F2.mul(correction_factor, axis=0)
    F3=F3.mul(correction_factor, axis=0)
    
    return F1,F2,F3

In [ ]:
# This function tries through the use of statistics to assign the EV consumption for each EV owner

def EV_load_profile(feederloads,Type_of_Day,truncated_normal_arrival,truncated_normal_departure):
    
    num_of_EVbus = (feederloads['EV']=='Yes').sum() # Find the name of EVs
    
    # Initialize arrays to store random consumption and battery capacity
    ran_consumption = np.zeros(num_of_EVbus)
    C_battery = np.zeros(num_of_EVbus)  

    # This is an predefined european sales statistics it can modified accordingly
    
    for i in range(num_of_EVbus):
        random_EV = np.random.rand()  # Generate a random value between 0 and 1
        if random_EV <= 0.237:
            ran_consumption[i] = 16.4
            C_battery[i] = 57.5
        elif random_EV > 0.237 and random_EV <= 0.444:
            ran_consumption[i] = 13.7
            C_battery[i] = 57.5
        elif random_EV > 0.444 and random_EV <= 0.542:
            ran_consumption[i] = 17.1
            C_battery[i] = 61.7
        elif random_EV > 0.542 and random_EV <= 0.638:
            ran_consumption[i] = 16.4
            C_battery[i] = 59
        elif random_EV > 0.638 and random_EV <= 0.715:
            ran_consumption[i] = 18.3
            C_battery[i] = 77
        elif random_EV > 0.715 and random_EV <= 0.785:
            ran_consumption[i] = 17.3
            C_battery[i] = 77
        elif random_EV > 0.785 and random_EV <= 0.849:
            ran_consumption[i] = 17.8
            C_battery[i] = 64
        elif random_EV > 0.849 and random_EV <= 0.905:
            ran_consumption[i] = 17.1
            C_battery[i] = 77
        elif random_EV > 0.905 and random_EV <= 0.956:
            ran_consumption[i] = 17.0
            C_battery[i] = 64.7
        elif random_EV > 0.956 and random_EV <= 1.0:
            ran_consumption[i] = 15.8
            C_battery[i] = 60
    # Initialize arrays to store arrival,departure times and distance
    random_arrival_times = []
    random_departure_times = []
    random_distance = []
    
    # random_value = np.random.choice(values_weekdays, p=probabilities_weekdays)
    # random_value = np.random.choice(values_weekends, p=probabilities_weekends)

    # By using the truncated distribution find the above mentioned three parameters 
    for i in range(num_of_EVbus):
        # Generate a random value and round to nearest half-hour
        random_time = round(truncated_normal_arrival.rvs() * 2)  
        random_arrival_times.append(random_time)
        random_time = round(truncated_normal_departure.rvs() *2) 
        random_departure_times.append(random_time)
        
        if Type_of_Day==0:
            # random_dist=round(np.random.normal(loc=mean_distance_wd, scale=std_dev_distance_wd)) #Using Trancated Normal Distribution 
            random_dist = round(np.random.choice(values_weekdays, p=probabilities_weekdays)) # Using probabilities 
        elif Type_of_Day==1:
            # random_dist=round(np.random.normal(loc=mean_distance_wk, scale=std_dev_distance_wk)) #Using Trancated Normal Distribution 
            random_dist = np.random.choice(values_weekends, p=probabilities_weekends) # Using probabilities 
        while (random_dist<0) or (random_dist>300):
            if Type_of_Day==0:
                random_dist=round(np.random.normal(loc=mean_distance_wd, scale=std_dev_distance_wd))
            elif Type_of_Day==1:
                random_dist=round(np.random.normal(loc=mean_distance_wk, scale=std_dev_distance_wk))
        
        random_distance.append(random_dist)
    
    C_charge = random_distance*ran_consumption/100 #kWh of charge that need to made for each EV
    
    time_of_charge = np.ceil(C_charge / feederloads[(feederloads['EV']=='Yes')]['EV_Power'] * 2) #No of 30-mins that need to charge
    
    # Create a time range for 30-minute intervals (48 periods to cover 24 hours)
    time_range = pd.date_range(start='00:00:00', end='23:59:59', freq='30min').time

    # Create the DataFrame to store power consumption at each time-interval
    df = pd.DataFrame(index=time_range)
    
    for ev in range(num_of_EVbus):
        column_name = f'EV{ev+1}'  # Name the column as EV1, EV2, EV3, etc.
        df[column_name] = 0  # Initialize all values to zero
        df[column_name] = df[column_name].astype(float)    
            
        # Convert the arrival time to the corresponding time index
        time_start = df.index[random_arrival_times[ev]]  # Retrieves the time corresponding to the 11th index
        time_end =  df.index[int(min(random_arrival_times[ev]+time_of_charge.iloc[ev],47))]
        df.loc[time_start:time_end, column_name] = feederloads[(feederloads['EV']=='Yes')]['EV_Power'].iloc[ev]
        if (int(random_arrival_times[ev]+time_of_charge.iloc[ev]))>48:
            time_start = df.index[0]
            time_end = df.index[int(time_of_charge.iloc[ev])-(48-int(random_arrival_times[ev]))]
            df.loc[time_start:time_end, column_name] = feederloads[(feederloads['EV']=='Yes')]['EV_Power'].iloc[ev]
    return df

In [ ]:
# This function assign the ev load profile on the basic load
def assign_ev_to_load(feederloads,Basic_Load,EV_Load):
    customer = 0 
    ev_customer = 0
    
    for i in range(len(feederloads)):
        if feederloads.iloc[i,:].loc['EV']=='Yes':
            Basic_Load.iloc[:,customer]=Basic_Load.iloc[:,customer] + EV_Load.iloc[:,ev_customer]
            customer=customer+1
            ev_customer=ev_customer+1
        elif feederloads.iloc[i,:].loc['Phase']!='None':
            customer=customer+1 
            
    return Basic_Load

In [ ]:
# This function assign the loads of each power flow in the network
def assign_loads(load_profile,pv_profile, df, FeederLoads, tan_theta, Q_Pv_profile):
    customer=0
    for i in range(len(FeederLoads)):
        if FeederLoads.iloc[i,:].loc['Type'] == 'None':
            bus_value = FeederLoads.iloc[i, :]['bus']
            df.loc[df['bus'] == bus_value, 'p_a_mw'] = 0
            df.loc[df['bus'] == bus_value, 'p_b_mw'] = 0
            df.loc[df['bus'] == bus_value, 'p_c_mw'] = 0
            df.loc[df['bus'] == bus_value, 'q_a_mvar'] = 0
            df.loc[df['bus'] == bus_value, 'q_b_mvar'] = 0
            df.loc[df['bus'] == bus_value, 'q_c_mvar'] = 0
            
        elif FeederLoads.iloc[i,:].loc['Type'] == '3-phase':
            bus_value = FeederLoads.iloc[i, :]['bus']
            df.loc[df['bus'] == bus_value, 'p_a_mw'] = load_profile.iloc[customer]/3 - pv_profile*FeederLoads.iloc[i,:]['PV_Power']/3
            df.loc[df['bus'] == bus_value, 'p_b_mw'] = load_profile.iloc[customer]/3 - pv_profile*FeederLoads.iloc[i,:]['PV_Power']/3
            df.loc[df['bus'] == bus_value, 'p_c_mw'] = load_profile.iloc[customer]/3 - pv_profile*FeederLoads.iloc[i,:]['PV_Power']/3
             
            df.loc[df['bus'] == bus_value, 'q_a_mvar'] = tan_theta*load_profile.iloc[customer]/3 + Q_Pv_profile*FeederLoads.iloc[i,:]['PV_Power']/3
            df.loc[df['bus'] == bus_value, 'q_b_mvar'] = tan_theta*load_profile.iloc[customer]/3 + Q_Pv_profile*FeederLoads.iloc[i,:]['PV_Power']/3
            df.loc[df['bus'] == bus_value, 'q_c_mvar'] = tan_theta*load_profile.iloc[customer]/3 + Q_Pv_profile*FeederLoads.iloc[i,:]['PV_Power']/3
            
            customer = customer + 1
                       
        elif FeederLoads.iloc[i,:].loc['Phase'] == 'A': 
            bus_value = FeederLoads.iloc[i, :].loc['bus']
            df.loc[df['bus'] == bus_value, 'p_a_mw'] =  load_profile.iloc[customer] - pv_profile*FeederLoads.iloc[i,:]['PV_Power']
            df.loc[df['bus'] == bus_value, 'p_b_mw'] =  0
            df.loc[df['bus'] == bus_value, 'p_c_mw'] =  0
            
            df.loc[df['bus'] == bus_value, 'q_a_mvar'] = tan_theta*load_profile.iloc[customer] + Q_Pv_profile*FeederLoads.iloc[i,:]['PV_Power']/3
            df.loc[df['bus'] == bus_value, 'q_b_mvar'] = 0
            df.loc[df['bus'] == bus_value, 'q_c_mvar'] = 0
            
            customer = customer + 1
            
        elif FeederLoads.iloc[i,:].loc['Phase'] == 'B': 
            bus_value = FeederLoads.iloc[i, :].loc['bus']
            df.loc[df['bus'] == bus_value, 'p_a_mw'] =  0
            df.loc[df['bus'] == bus_value, 'p_b_mw'] =  load_profile.iloc[customer] - pv_profile*FeederLoads.iloc[i,:]['PV_Power']
            df.loc[df['bus'] == bus_value, 'p_c_mw'] = 0
            
            df.loc[df['bus'] == bus_value, 'q_a_mvar'] = 0
            df.loc[df['bus'] == bus_value, 'q_b_mvar'] = tan_theta*load_profile.iloc[customer] + Q_Pv_profile*FeederLoads.iloc[i,:]['PV_Power']/3
            df.loc[df['bus'] == bus_value, 'q_c_mvar'] = 0
            
            customer = customer + 1

        elif FeederLoads.iloc[i,:].loc['Phase'] == 'C': 
            bus_value = FeederLoads.iloc[i, :].loc['bus']
            df.loc[df['bus'] == bus_value, 'p_a_mw'] =  0
            df.loc[df['bus'] == bus_value, 'p_b_mw'] =  0
            df.loc[df['bus'] == bus_value, 'p_c_mw'] =  load_profile.iloc[customer] - pv_profile*FeederLoads.iloc[i,:]['PV_Power']
            
            df.loc[df['bus'] == bus_value, 'q_a_mvar'] = 0
            df.loc[df['bus'] == bus_value, 'q_b_mvar'] = 0
            df.loc[df['bus'] == bus_value, 'q_c_mvar'] = tan_theta*load_profile.iloc[customer] + Q_Pv_profile*FeederLoads.iloc[i,:]['PV_Power']/3
            
            customer = customer + 1
                   
    return df

In [ ]:
# This function return the days in year where the month is the input month that will given
import calendar

def get_days_in_month(month, year=2024):
    days_in_month = []
    
    # Check if it's a leap year
    is_leap = calendar.isleap(year)
    
    # List of days in each month
    month_days = [31, 29 if is_leap else 28, 31, 30, 31, 30,
                  31, 31, 30, 31, 30, 31]
    
    # Validate the month input
    if month < 1 or month > 12:
        raise ValueError("Month must be between 1 and 12")
    
    # Calculate cumulative days up to the previous month
    cumulative_days = sum(month_days[:month-1])
    
    # Add day of year for each day in the month
    for day in range(1, month_days[month-1] + 1):
        day_of_year = cumulative_days + day
        days_in_month.append(day_of_year)
    
    return days_in_month


In [ ]:
# This function use all the above details and run a quasi-dynamic simulation for the whole time-frame (24h with 30-min interval)
def quasi_sim(net,load_profile, pv_profile,month_name,Type_of_Day,tan_theta,three_phase_percentage,PV_percentage,EV_percentage,Total_loads_per_feeder,PV_prob,Q_Pv_profile):
    month_to_number = {
    "January": 1, "February": 2, "March": 3, "April": 4,
    "May": 5, "June": 6, "July": 7, "August": 8,
    "September": 9, "October": 10, "November": 11, "December": 12
}   
    time_range = pd.date_range(start='00:00:00', end='23:59:59', freq='30min').time
    results_phaseA_min=pd.DataFrame()
    results_phaseB_min=pd.DataFrame()
    results_phaseC_min=pd.DataFrame()
    results_phaseA_max=pd.DataFrame()
    results_phaseB_max=pd.DataFrame()
    results_phaseC_max=pd.DataFrame()
    results_transformer=pd.DataFrame()
    results_lines=pd.DataFrame()
    # Convert month name to month number
    month_number = month_to_number.get(month_name)
    days_of_year_in_july = get_days_in_month(month_number)
    load_profile = load_profile[:,days_of_year_in_july]
    Simulate_Column = month_number-1
    pv_profile = pv_profile.iloc[:, Simulate_Column]
    pv_q_profile = Q_Pv_profile.iloc[:, Simulate_Column]
    Feeder1Loads = assign_phase_type(net.asymmetric_load, three_phase_percentage,PV_percentage, EV_percentage, Total_loads_per_feeder,'F1',values_one_phase,probabilities_one_phase,values_three_phase,probabilities_three_phase)
    Feeder2Loads = assign_phase_type(net.asymmetric_load, three_phase_percentage,PV_percentage, EV_percentage, Total_loads_per_feeder,'F2',values_one_phase,probabilities_one_phase,values_three_phase,probabilities_three_phase)
    Feeder3Loads = assign_phase_type(net.asymmetric_load, three_phase_percentage,PV_percentage, EV_percentage, Total_loads_per_feeder,'F3',values_one_phase,probabilities_one_phase,values_three_phase,probabilities_three_phase)
    F1_Basic_Load = Basic_Load_profile(Feeder1Loads,Type_of_Day,load_profile)
    F2_Basic_Load = Basic_Load_profile(Feeder2Loads,Type_of_Day,load_profile)
    F3_Basic_Load = Basic_Load_profile(Feeder3Loads,Type_of_Day,load_profile)
    
    F1_EV_load = EV_load_profile(Feeder1Loads,Type_of_Day,truncated_normal_arrival,truncated_normal_departure)
    F2_EV_load = EV_load_profile(Feeder2Loads,Type_of_Day,truncated_normal_arrival,truncated_normal_departure)
    F3_EV_load = EV_load_profile(Feeder3Loads,Type_of_Day,truncated_normal_arrival,truncated_normal_departure)
    
    F1_load_profile = assign_ev_to_load(Feeder1Loads,F1_Basic_Load,F1_EV_load)
    F2_load_profile = assign_ev_to_load(Feeder2Loads,F2_Basic_Load,F2_EV_load)
    F3_load_profile = assign_ev_to_load(Feeder3Loads,F3_Basic_Load,F3_EV_load)
    day_results = {}
    bus_results= {}
    for i in range(len(pv_profile)):
        net.asymmetric_load= assign_loads(F1_load_profile.iloc[i,:]/1000,(pv_profile.iloc[i]/1000), net.asymmetric_load, Feeder1Loads, tan_theta, pv_q_profile.iloc[i]/1000)
        net.asymmetric_load= assign_loads(F2_load_profile.iloc[i,:]/1000,(pv_profile.iloc[i]/1000), net.asymmetric_load, Feeder2Loads, tan_theta, pv_q_profile.iloc[i]/1000)
        net.asymmetric_load= assign_loads(F3_load_profile.iloc[i,:]/1000,(pv_profile.iloc[i]/1000), net.asymmetric_load, Feeder3Loads, tan_theta, pv_q_profile.iloc[i]/1000)
        
        pp.runpp_3ph(net)
        
        bus_results[i] = net.res_bus_3ph
        
        results_phaseA_min = pd.concat([results_phaseA_min, pd.DataFrame(net.res_bus_3ph.iloc[net.res_bus_3ph['vm_a_pu'].idxmin(),:]).T], ignore_index=False)
        results_phaseB_min = pd.concat([results_phaseB_min, pd.DataFrame(net.res_bus_3ph.iloc[net.res_bus_3ph['vm_b_pu'].idxmin(),:]).T], ignore_index=False)        
        results_phaseC_min = pd.concat([results_phaseC_min, pd.DataFrame(net.res_bus_3ph.iloc[net.res_bus_3ph['vm_c_pu'].idxmin(),:]).T], ignore_index=False)

        results_phaseA_max = pd.concat([results_phaseA_max, pd.DataFrame(net.res_bus_3ph.iloc[net.res_bus_3ph['vm_a_pu'].idxmax(),:]).T], ignore_index=False)
        results_phaseB_max = pd.concat([results_phaseB_max, pd.DataFrame(net.res_bus_3ph.iloc[net.res_bus_3ph['vm_b_pu'].idxmax(),:]).T], ignore_index=False)
        results_phaseC_max = pd.concat([results_phaseC_max, pd.DataFrame(net.res_bus_3ph.iloc[net.res_bus_3ph['vm_c_pu'].idxmax(),:]).T], ignore_index=False)
        
        results_transformer = pd.concat([results_transformer, pd.DataFrame(net.res_trafo_3ph.iloc[0,:]).T], ignore_index=False)
        results_lines = pd.concat([results_lines, pd.DataFrame(net.res_line_3ph.iloc[net.res_line_3ph.iloc[0:3]['loading_percent'].idxmax(),:]).T], ignore_index=False)
        
    results_phaseA_min['name'] = net.bus.loc[results_phaseA_min.index,'name']
    results_phaseA_min.index = time_range
    results_phaseB_min['name'] = net.bus.loc[results_phaseB_min.index,'name']
    results_phaseB_min.index = time_range 
    results_phaseC_min['name'] = net.bus.loc[results_phaseC_min.index,'name']
    results_phaseC_min.index = time_range
    results_phaseA_max['name'] = net.bus.loc[results_phaseA_max.index,'name']
    results_phaseA_max.index = time_range    
    results_phaseB_max['name'] = net.bus.loc[results_phaseB_max.index,'name']
    results_phaseB_max.index = time_range
    results_phaseC_max['name'] = net.bus.loc[results_phaseC_max.index,'name']
    results_phaseC_max.index = time_range
    results_transformer['name'] = net.trafo.loc[results_transformer.index,'name']
    results_transformer.index = time_range
    # results_lines['name'] = net.trafo.loc[results_lines.index,'name']
    results_lines.index = time_range
    
    day_results[f'Min_PhaseA'] = results_phaseA_min
    day_results[f'Min_PhaseB'] = results_phaseB_min
    day_results[f'Min_PhaseC'] = results_phaseC_min
    day_results[f'Max_PhaseA'] = results_phaseA_max
    day_results[f'Max_PhaseB'] = results_phaseB_max
    day_results[f'Max_PhaseC'] = results_phaseC_max
    day_results[f'Transformer'] = results_transformer
    day_results[f'bus_results'] = bus_results
    day_results[f'F1_EV_Profile'] = F1_EV_load
    day_results[f'F2_EV_Profile'] = F2_EV_load
    day_results[f'F3_EV_Profile'] = F3_EV_load
    day_results[f'Lines'] = results_lines
    
    
    return  day_results